In [1]:
# ! python3 -m pip install xlrd==1.2.0
# ! python3 -m pip install openpyxl

In [2]:
import pandas as pd

In [3]:
file = 'ze_delivery_data_base.xlsx'
raw = f'../Datalake/Raw/{file}'

In [4]:
sheet1 = 'Order rating DB'
sheet2 = 'Bad Rating reasons'
sheet3 = 'Rating e-mail data'

In [5]:
df_order = pd.read_excel(raw, engine = 'openpyxl', sheet_name = sheet1)
df_bad_rating = pd.read_excel(raw, engine = 'openpyxl', sheet_name= sheet2)
df_email_rating = pd.read_excel(raw, engine = 'openpyxl', sheet_name= sheet3)

## ETL - Order rating DB

In [6]:
df_order.shape

(89351, 13)

In [7]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89351 entries, 0 to 89350
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order ID        11259 non-null  float64       
 1   Order Number    11259 non-null  float64       
 2   Week            11259 non-null  float64       
 3   Date & Time     11259 non-null  datetime64[ns]
 4   Comment         127 non-null    object        
 5   User            11257 non-null  object        
 6   Gross Revenue   11259 non-null  float64       
 7   Payment Method  5990 non-null   object        
 8   POC ID          11259 non-null  float64       
 9   Rating          5134 non-null   float64       
 10  Reasons         0 non-null      float64       
 11  Shipping Type   11259 non-null  object        
 12  State           11239 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(5)
memory usage: 8.9+ MB


In [8]:
df_order.head()

,Order ID,Order Number,Week,Date & Time,Comment,User,Gross Revenue,Payment Method,POC ID,Rating,Reasons,Shipping Type,State
0,191357.0,14027.0,1.0,2017-01-01 10:11:01,NaN,user7gmail.com,105.120,NaN,26.0,5.0,NaN,receber,SP
1,191361.0,14031.0,1.0,2017-01-01 11:39:28,NaN,user11gmail.com,70.848,NaN,30.0,5.0,NaN,receber,SP
2,191362.0,14032.0,1.0,2017-01-01 12:22:29,NaN,user12gmail.com,62.304,NaN,26.0,NaN,NaN,receber,SP
3,191384.0,14054.0,1.0,2017-01-01 15:35:10,NaN,user33gmail.com,48.816,NaN,29.0,1.0,NaN,receber,SP
4,191385.0,14055.0,1.0,2017-01-01 16:02:38,NaN,user34gmail.com,70.848,NaN,30.0,5.0,NaN,receber,SP


In [9]:
def clean_column(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('&', '')
    df.columns = df.columns.str.replace(' ', '_', regex=True)
    df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    return df

df_order = clean_column(df_order)

In [10]:
df_order.head()

,order_id,order_number,week,date__time,comment,user,gross_revenue,payment_method,poc_id,rating,reasons,shipping_type,state
0,191357.0,14027.0,1.0,2017-01-01 10:11:01,NaN,user7gmail.com,105.120,NaN,26.0,5.0,NaN,receber,SP
1,191361.0,14031.0,1.0,2017-01-01 11:39:28,NaN,user11gmail.com,70.848,NaN,30.0,5.0,NaN,receber,SP
2,191362.0,14032.0,1.0,2017-01-01 12:22:29,NaN,user12gmail.com,62.304,NaN,26.0,NaN,NaN,receber,SP
3,191384.0,14054.0,1.0,2017-01-01 15:35:10,NaN,user33gmail.com,48.816,NaN,29.0,1.0,NaN,receber,SP
4,191385.0,14055.0,1.0,2017-01-01 16:02:38,NaN,user34gmail.com,70.848,NaN,30.0,5.0,NaN,receber,SP


### Clean empty rows

In [11]:
df_order = df_order[df_order.order_id.notna()]
df_order.shape

(11259, 13)

### Payment_method

In [12]:
df_order.payment_method.unique()

array([nan, 'Cartão de Débito', 'Cartão de Crédito', 'Dinheiro', 'Débito',
       'Crédito'], dtype=object)

In [13]:
df_order.groupby('payment_method').count()

,order_id,order_number,week,date__time,comment,user,gross_revenue,poc_id,rating,reasons,shipping_type,state
payment_method,,,,,,,,,,,,
Cartão de Crédito,1029,1029,1029,1029,17,1029,1029,1029,426,0,1029,1021
Cartão de Débito,866,866,866,866,7,866,866,866,340,0,866,860
Crédito,1086,1086,1086,1086,28,1086,1086,1086,408,0,1086,1086
Dinheiro,2023,2023,2023,2023,42,2021,2023,2023,743,0,2023,2017
Débito,986,986,986,986,33,986,986,986,417,0,986,986


In [14]:
df_order['payment_method'] = df_order.payment_method.replace(['Cartão de Débito', 'Cartão de Crédito'], ['Débito', 'Crédito'])
df_order.groupby('payment_method').count()

,order_id,order_number,week,date__time,comment,user,gross_revenue,poc_id,rating,reasons,shipping_type,state
payment_method,,,,,,,,,,,,
Crédito,2115,2115,2115,2115,45,2115,2115,2115,834,0,2115,2107
Dinheiro,2023,2023,2023,2023,42,2021,2023,2023,743,0,2023,2017
Débito,1852,1852,1852,1852,40,1852,1852,1852,757,0,1852,1846


In [15]:
df_order['payment_method'] = df_order.payment_method.fillna('sem informação')

### State

In [16]:
df_order.state.unique()

array(['SP', 'RJ', 'MG', 'CE', 'PR', 'Campinas', 'São Paulo', 'Fortaleza',
       'São José do Rio Preto', 'Rio de Janeiro', 'S',
       'São Bernardo do Campo', 'Diadema', 'Belo Horizonte', nan,
       'Santo André', 'Curitiba', 'Betim', 'São José dos Campos',
       'Nova Iguaçu', 'Niter', 'Sao Paulo', 'São Caetano do Sul',
       'Nova Igua', 'Niterói', 'Guarulhos', 'Embu', 'Poços de Caldas',
       'Santo Andr', 'Itaquaquecetuba', 'Belford Roxo', 'Mesquita',
       'Contagem', 'Colombo', 'undefined', 'Osasco', 'Carapicuíba',
       'Embu das Artes', 'Taboão da Serra', 'Ferraz de Vasconcelos',
       'Pinhais', 'São Carlos', 'Maracanaú', 'Barueri', 'Nova Lima'],
      dtype=object)

In [17]:
df_order.groupby('state').count()

,order_id,order_number,week,date__time,comment,user,gross_revenue,payment_method,poc_id,rating,reasons,shipping_type
state,,,,,,,,,,,,
Barueri,1,1,1,1,0,1,1,1,1,1,0,1
Belford Roxo,48,48,48,48,2,48,48,48,48,21,0,48
Belo Horizonte,345,345,345,345,13,345,345,345,345,141,0,345
Betim,53,53,53,53,0,53,53,53,53,29,0,53
CE,77,77,77,77,0,77,77,77,77,49,0,77
Campinas,229,229,229,229,2,229,229,229,229,88,0,229
Carapicuíba,1,1,1,1,0,1,1,1,1,0,0,1
Colombo,5,5,5,5,0,5,5,5,5,1,0,5
Contagem,26,26,26,26,3,26,26,26,26,20,0,26


In [18]:
df_order['state'] = df_order.state.replace(['undefined', 'S', 'Embu', 'Niter', 'Nova Igua', 'Santo Andr', 'Sao Paulo'], 
                                           ['Sem cidade', 'Sem cidade', 'Embu das Artes', 'Niterói', 'Nova Iguaçu', 'Santo André', 'São Paulo'])

In [19]:
df_order.groupby('state').count()

,order_id,order_number,week,date__time,comment,user,gross_revenue,payment_method,poc_id,rating,reasons,shipping_type
state,,,,,,,,,,,,
Barueri,1,1,1,1,0,1,1,1,1,1,0,1
Belford Roxo,48,48,48,48,2,48,48,48,48,21,0,48
Belo Horizonte,345,345,345,345,13,345,345,345,345,141,0,345
Betim,53,53,53,53,0,53,53,53,53,29,0,53
CE,77,77,77,77,0,77,77,77,77,49,0,77
Campinas,229,229,229,229,2,229,229,229,229,88,0,229
Carapicuíba,1,1,1,1,0,1,1,1,1,0,0,1
Colombo,5,5,5,5,0,5,5,5,5,1,0,5
Contagem,26,26,26,26,3,26,26,26,26,20,0,26


### shipping type

In [20]:
df_order.shipping_type.unique()

array(['receber', 'retirar', 'agendar'], dtype=object)

### Reasons

In [21]:
df_order.reasons.unique()

array([nan])

In [22]:
df_order.columns.values

array(['order_id', 'order_number', 'week', 'date__time', 'comment',
       'user', 'gross_revenue', 'payment_method', 'poc_id', 'rating',
       'reasons', 'shipping_type', 'state'], dtype=object)

In [23]:
df_order = df_order[['order_id', 'order_number', 'week', 'date__time', 'comment', 'user', 'gross_revenue', 'payment_method', 'poc_id', 'rating', 'shipping_type', 'state']]

In [24]:
df_order.columns.values

array(['order_id', 'order_number', 'week', 'date__time', 'comment',
       'user', 'gross_revenue', 'payment_method', 'poc_id', 'rating',
       'shipping_type', 'state'], dtype=object)

### Rating

In [25]:
df_order.rating.unique()

array([ 5., nan,  1.,  3.,  4.,  2., -1.])

In [26]:
df_order['rating'] = df_order.rating.fillna('0')

In [27]:
df_order.rating.unique()

array([5.0, '0', 1.0, 3.0, 4.0, 2.0, -1.0], dtype=object)

### comment

In [28]:
df_order.head()

,order_id,order_number,week,date__time,comment,user,gross_revenue,payment_method,poc_id,rating,shipping_type,state
0,191357.0,14027.0,1.0,2017-01-01 10:11:01,NaN,user7gmail.com,105.120,sem informação,26.0,5,receber,SP
1,191361.0,14031.0,1.0,2017-01-01 11:39:28,NaN,user11gmail.com,70.848,sem informação,30.0,5,receber,SP
2,191362.0,14032.0,1.0,2017-01-01 12:22:29,NaN,user12gmail.com,62.304,sem informação,26.0,0,receber,SP
3,191384.0,14054.0,1.0,2017-01-01 15:35:10,NaN,user33gmail.com,48.816,sem informação,29.0,1,receber,SP
4,191385.0,14055.0,1.0,2017-01-01 16:02:38,NaN,user34gmail.com,70.848,sem informação,30.0,5,receber,SP


In [29]:
df_order.comment.unique()

array([nan, 'Adorei esse aplicativo', 'Quero receber os descontos',
       'Sobre o site, gostaria de sugerir que ao fazer o login, você não precise mais perguntar meu endereço.',
       'O cara demorou um pouco para entregar.',
       'Rápido e muuuuito prático', 'Muito bom, top', 'Adorei', 'Ddd',
       'muiiiitoooo foooddaaaaa!!!!!!!11',
       'Primeira vez q peço é a cerveja veio quente pra caramba. Passei vergonha com os convidados. Dificilmente pedirei outra vez',
       'Muito bom e rapido', 'Muitos rápidos, excelente!!!',
       'eu achei incrível, porém achei o frete caro pra quem mora perto como eu rs, tipo duas ruas para baixo do local de entrega',
       'Ainda não recebi', 'Não recebi', 'Excelente', 'Rápida e gelada',
       'buenazoooooooo', 'Tudo certo',
       'o site deveria lembrar meus ultimos endereços e sugeriro  envio  para eles',
       'apenas o frete que saiu um pouco alto... mas a entrega muito rápida excelente serviço!!',
       'Não entregou', 'Excelente!',

In [30]:
df_order['comment'] = df_order.comment.fillna('sem comentario')

### user

In [31]:
df_order.groupby('user').count()

,order_id,order_number,week,date__time,comment,gross_revenue,payment_method,poc_id,rating,shipping_type,state
user,,,,,,,,,,,
user10001gmail.com,1,1,1,1,1,1,1,1,1,1,1
user10010gmail.com,2,2,2,2,2,2,2,2,2,2,2
user10015gmail.com,2,2,2,2,2,2,2,2,2,2,2
user10016gmail.com,1,1,1,1,1,1,1,1,1,1,1
user10017gmail.com,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
user9991gmail.com,2,2,2,2,2,2,2,2,2,2,2
user9992gmail.com,1,1,1,1,1,1,1,1,1,1,1
user9997gmail.com,1,1,1,1,1,1,1,1,1,1,1


In [32]:
df_order.head()

,order_id,order_number,week,date__time,comment,user,gross_revenue,payment_method,poc_id,rating,shipping_type,state
0,191357.0,14027.0,1.0,2017-01-01 10:11:01,sem comentario,user7gmail.com,105.120,sem informação,26.0,5,receber,SP
1,191361.0,14031.0,1.0,2017-01-01 11:39:28,sem comentario,user11gmail.com,70.848,sem informação,30.0,5,receber,SP
2,191362.0,14032.0,1.0,2017-01-01 12:22:29,sem comentario,user12gmail.com,62.304,sem informação,26.0,0,receber,SP
3,191384.0,14054.0,1.0,2017-01-01 15:35:10,sem comentario,user33gmail.com,48.816,sem informação,29.0,1,receber,SP
4,191385.0,14055.0,1.0,2017-01-01 16:02:38,sem comentario,user34gmail.com,70.848,sem informação,30.0,5,receber,SP


## ETL - Bad Rating reasons

In [33]:
df_bad_rating.shape

(21718, 5)

In [34]:
df_bad_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21718 entries, 0 to 21717
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Order ID         215 non-null    float64       
 1   Order Number     215 non-null    float64       
 2   rating_datetime  215 non-null    datetime64[ns]
 3   rating_reasons   97 non-null     object        
 4   Rating           203 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 848.5+ KB


In [35]:
df_bad_rating.head()

,Order ID,Order Number,rating_datetime,rating_reasons,Rating
0,140.0,3234121.0,2017-07-06 00:51:32,NaN,4.0
1,4235.0,78020343.0,2017-09-14 20:24:15,mais de 1 hora,4.0
2,4405.0,27845708.0,2017-07-07 19:45:16,NaN,4.0
3,39488.0,75122867.0,2017-08-23 19:51:42,NaN,4.0
4,39589.0,52984791.0,2017-07-07 23:00:47,NaN,1.0


In [36]:
df_bad_rating = clean_column(df_bad_rating)

In [37]:
df_bad_rating.head()

,order_id,order_number,rating_datetime,rating_reasons,rating
0,140.0,3234121.0,2017-07-06 00:51:32,NaN,4.0
1,4235.0,78020343.0,2017-09-14 20:24:15,mais de 1 hora,4.0
2,4405.0,27845708.0,2017-07-07 19:45:16,NaN,4.0
3,39488.0,75122867.0,2017-08-23 19:51:42,NaN,4.0
4,39589.0,52984791.0,2017-07-07 23:00:47,NaN,1.0


In [38]:
df_bad_rating.order_id.isna().sum()

21503

In [39]:
df_bad_rating.order_id.notna().sum()

215

In [40]:
df_bad_rating = df_bad_rating[df_bad_rating.order_id.notna()]
df_bad_rating.shape

(215, 5)

### Rating reasons

In [41]:
df_bad_rating.rating_reasons.unique()

array([nan, 'mais de 1 hora', 'faltou produto', 'outra coisa',
       'cerveja quente', 'problema no pagamento', 'serviço ruim',
       'Não entregue'], dtype=object)

In [42]:
df_bad_rating['rating_reasons'] = df_bad_rating.rating_reasons.fillna('sem informacao motivo')

In [43]:
df_bad_rating.head()

,order_id,order_number,rating_datetime,rating_reasons,rating
0,140.0,3234121.0,2017-07-06 00:51:32,sem informacao motivo,4.0
1,4235.0,78020343.0,2017-09-14 20:24:15,mais de 1 hora,4.0
2,4405.0,27845708.0,2017-07-07 19:45:16,sem informacao motivo,4.0
3,39488.0,75122867.0,2017-08-23 19:51:42,sem informacao motivo,4.0
4,39589.0,52984791.0,2017-07-07 23:00:47,sem informacao motivo,1.0


In [44]:
df_bad_rating.rating_reasons.unique()

array(['sem informacao motivo', 'mais de 1 hora', 'faltou produto',
       'outra coisa', 'cerveja quente', 'problema no pagamento',
       'serviço ruim', 'Não entregue'], dtype=object)

### Rating

In [45]:
df_bad_rating.rating.unique()

array([ 4.,  1.,  3.,  2., nan])

In [46]:
df_bad_rating.head()

,order_id,order_number,rating_datetime,rating_reasons,rating
0,140.0,3234121.0,2017-07-06 00:51:32,sem informacao motivo,4.0
1,4235.0,78020343.0,2017-09-14 20:24:15,mais de 1 hora,4.0
2,4405.0,27845708.0,2017-07-07 19:45:16,sem informacao motivo,4.0
3,39488.0,75122867.0,2017-08-23 19:51:42,sem informacao motivo,4.0
4,39589.0,52984791.0,2017-07-07 23:00:47,sem informacao motivo,1.0


In [47]:
df_bad_rating['rating'] = df_bad_rating.rating.fillna('0')

In [48]:
df_bad_rating.head()

,order_id,order_number,rating_datetime,rating_reasons,rating
0,140.0,3234121.0,2017-07-06 00:51:32,sem informacao motivo,4
1,4235.0,78020343.0,2017-09-14 20:24:15,mais de 1 hora,4
2,4405.0,27845708.0,2017-07-07 19:45:16,sem informacao motivo,4
3,39488.0,75122867.0,2017-08-23 19:51:42,sem informacao motivo,4
4,39589.0,52984791.0,2017-07-07 23:00:47,sem informacao motivo,1


In [49]:
df_bad_rating.rating.unique()

array([4.0, 1.0, 3.0, 2.0, '0'], dtype=object)

In [50]:
df_bad_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 0 to 214
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   order_id         215 non-null    float64       
 1   order_number     215 non-null    float64       
 2   rating_datetime  215 non-null    datetime64[ns]
 3   rating_reasons   215 non-null    object        
 4   rating           215 non-null    object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 10.1+ KB


## ETL - Rating e-mail data

In [51]:
df_email_rating.head(10)

,Origem de acesso ao site,Mês do ano,Visitas no Site,Transações
0,EmailAvaliacao / EmailTransacional,11,91,29
1,EmailAvaliacao / EmailTransacional,10,253,58
2,EmailAvaliacao / EmailTransacional,9,514,87
3,EmailAvaliacao / EmailTransacional,8,819,146
4,EmailAvaliacao / EmailTransacional,7,1396,301
5,EmailAvaliacao / EmailTransacional,6,1756,200
6,EmailAvaliacao / EmailTransacional,5,2379,169
7,EmailAvaliacao / EmailTransacional,4,2574,175
8,EmailAvaliacao / EmailTransacional,3,1559,112
9,EmailAvaliacao / EmailTransacional,2,1176,66


## Join

In [52]:
df_bad_rating = df_bad_rating[['order_id', 'order_number', 'rating_datetime', 'rating_reasons']]
df_bad_rating.shape

(215, 4)

In [53]:
df_order.shape

(11259, 12)

In [54]:
data = df_order.merge(df_bad_rating, on=['order_id', 'order_number'], how='left')

## Date

In [55]:
data['date_time'] = pd.to_datetime(data['date__time'], format = '%Y-%m-%d %H:%M:%S')
data['date_week_day'] = data.date__time.dt.weekday
data['date_day'] = data.date__time.dt.day
data['date_month'] = data.date__time.dt.month
data['date_day_of_year'] = data.date__time.dt.dayofyear
data['date_hour'] = data.date__time.dt.hour
data['date_day_month'] = data['date_time'].dt.strftime('%m-%d')

In [56]:
data = data[data.columns[~data.columns.isin(['date__time'])]]

In [57]:
data.head()

,order_id,order_number,week,comment,user,gross_revenue,payment_method,poc_id,rating,shipping_type,state,rating_datetime,rating_reasons,date_time,date_week_day,date_day,date_month,date_day_of_year,date_hour,date_day_month
0,191357.0,14027.0,1.0,sem comentario,user7gmail.com,105.120,sem informação,26.0,5,receber,SP,NaT,NaN,2017-01-01 10:11:01,6,1,1,1,10,01-01
1,191361.0,14031.0,1.0,sem comentario,user11gmail.com,70.848,sem informação,30.0,5,receber,SP,NaT,NaN,2017-01-01 11:39:28,6,1,1,1,11,01-01
2,191362.0,14032.0,1.0,sem comentario,user12gmail.com,62.304,sem informação,26.0,0,receber,SP,NaT,NaN,2017-01-01 12:22:29,6,1,1,1,12,01-01
3,191384.0,14054.0,1.0,sem comentario,user33gmail.com,48.816,sem informação,29.0,1,receber,SP,NaT,NaN,2017-01-01 15:35:10,6,1,1,1,15,01-01
4,191385.0,14055.0,1.0,sem comentario,user34gmail.com,70.848,sem informação,30.0,5,receber,SP,NaT,NaN,2017-01-01 16:02:38,6,1,1,1,16,01-01


## Load

In [58]:
refined = '../Datalake/Refined'
sheet1 = 'order_rating_db'
sheet2 = 'bad_rating_reasons'
sheet3 = 'rating_email_data'

In [59]:
df_order.to_csv(f'{refined}/{sheet1}.csv', sep=',', encoding='utf-8', index=False)
df_bad_rating.to_csv(f'{refined}/{sheet2}.csv', sep=',', encoding='utf-8', index=False)
df_email_rating.to_csv(f'{refined}/{sheet3}.csv', sep=',', encoding='utf-8', index=False)
data.to_csv(f'{refined}/refined_data.csv', sep=',', encoding='utf-8', index=False)

In [60]:
data.to_csv(f'{refined}/refined_data.csv', sep=',', encoding='utf-8', index=False)